In [121]:
import sys
from pathlib import Path
src_path = Path('../src').resolve()
sys.path.append(str(src_path))
src_path = Path('../../BOTalign').resolve()
sys.path.append(str(src_path))

from utils_BO import align_BO

from aspire.volume import Volume
from aspire.utils.rotation import Rotation
import mrcfile
import numpy as np 
import numpy.linalg as LA 
from scipy.io import loadmat, savemat


In [122]:
def Rz(th):
    """
    Rotation around the z axis 
    :param th: The rotation angle 
    :return: The 3x3 rotation matrix rotating a vector around z axis by th
    """
    return np.array([
        [np.cos(th), -np.sin(th), 0], 
        [np.sin(th), np.cos(th), 0],
        [0, 0, 1]
    ])


def Ry(th):
    """
    Rotation around the y axis 
    :param th: The rotation angle 
    :return: The 3x3 rotation matrix rotating a vector around y axis by th
    """    
    return np.array([
        [np.cos(th), 0, np.sin(th)], 
        [0, 1, 0],
        [-np.sin(th), 0, np.cos(th)]
    ])

In [123]:
with mrcfile.open('../data/emd_34948.map') as mrc:
    data = mrc.data


data = data/LA.norm(data.flatten())
Vol = Volume(data)
ds_res = 64 
vol = Vol.downsample(ds_res)


para =['wemd',64,150,True] 
Rot = Rz(1)@Ry(1)@Rz(3)
Rot = np.array(Rot, dtype=np.float32) 
Rot = Rotation(Rot)
vol_flip = vol.flip()
_vol = vol_flip.rotate(Rot)
_vol_flip = _vol.flip()
_, R_est1 = align_BO(vol,_vol,para,reflect=False)
_, R_est2 = align_BO(vol,_vol_flip,para,reflect=False)

In [124]:
[U,S,Vh] = LA.svd(R_est1)
S

array([1., 1., 1.], dtype=float32)

In [125]:
[U,S,Vh] = LA.svd(R_est2)
S

array([1.        , 1.        , 0.99999994], dtype=float32)

In [126]:
_vol_est = _vol.rotate(Rotation(R_est1))
LA.norm(_vol_est.asnumpy().flatten() - vol.asnumpy().flatten())

0.16751678

In [127]:
_vol_est = _vol_flip.rotate(Rotation(R_est2))
LA.norm(_vol_est.asnumpy().flatten() - vol.asnumpy().flatten())

0.09137104